# SHACL experiments

In [5]:
from typing import Callable

from pyshacl import validate
from rdflib import Graph, URIRef, Literal, Namespace, SH, RDF, RDFS, BNode, XSD

SCHEMA = Namespace("https://schema.org/")

g = Graph(bind_namespaces="none")
g.bind("schema", SCHEMA)
g.bind("sh", SH)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)

## Defining a type: PromptTemplate

In [6]:
prompt_template_type = URIRef("http://example.org/type/prompt-template")

## Constraining a type: PromptTemplateShape

In [7]:
prompt_template_shape = URIRef("http://example.org/shape/prompt-template")

g.add((prompt_template_shape, RDF.type, SH.NodeShape))
g.add((prompt_template_shape, SH.targetClass, prompt_template_type))

def add_name_property(shape) -> Callable[[Graph], Graph]:
    def wrapper(g: Graph) -> Graph:
        name_property = BNode()
        g.add((shape, SH.property, name_property))
        g.add((name_property, SH.path, SCHEMA.name))
        g.add((name_property, SH.datatype, XSD.string))
        g.add((name_property, SH.minCount, Literal(1)))
        g.add((name_property, SH.maxCount, Literal(1)))
        return g
    return wrapper

# Property: require a name added to the PromptTemplate
g = add_name_property(prompt_template_shape)(g)

# name_property = BNode()
# g.add((prompt_template_shape, SH.property, name_property))
# g.add((name_property, SH.path, SCHEMA.name))
# g.add((name_property, SH.datatype, XSD.string))
# g.add((name_property, SH.minCount, Literal(1)))
# g.add((name_property, SH.maxCount, Literal(1)))

def add_description_property(shape) -> Callable[[Graph], Graph]:
    def wrapper(g: Graph) -> Graph:
        description_property = BNode()
        g.add((shape, SH.property, description_property))
        g.add((description_property, SH.path, SCHEMA.description))
        g.add((description_property, SH.datatype, XSD.string))
        g.add((description_property, SH.minCount, Literal(1)))
        g.add((description_property, SH.maxCount, Literal(1)))
        return g
    return wrapper

# Property: require a description
g = add_description_property(prompt_template_shape)(g)

# description_property = BNode()
# g.add((prompt_template_shape, SH.property, description_property))
# g.add((description_property, SH.path, SCHEMA.description))
# g.add((description_property, SH.datatype, XSD.string))
# g.add((description_property, SH.minCount, Literal(1)))
# g.add((description_property, SH.maxCount, Literal(1)))

## Defining an instance: PromptTemplate

In [8]:
prompt_template_instance = URIRef("http://example.org/prompt-template/1")

g.add((prompt_template_instance, RDF.type, prompt_template_type))
g.add((prompt_template_instance, SCHEMA.name, Literal("Some kind of prompt")))
g.add((prompt_template_instance, SCHEMA.description, Literal("Some kind of template named: {{name}}")))

<Graph identifier=N46ccd1702bca496391c138caaa6c3097 (<class 'rdflib.graph.Graph'>)>

## Verification

Visual verification of a serialized graph in JSON-LD format.

In [9]:
jsonld = g.serialize(format="json-ld", indent=2, auto_compact=True)
print(jsonld)

{
  "@context": {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "https://schema.org/",
    "sh": "http://www.w3.org/ns/shacl#",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@graph": [
    {
      "@id": "http://example.org/shape/prompt-template",
      "@type": "sh:NodeShape",
      "sh:property": [
        {
          "@id": "_:N056ddae112d243938488a3369ef22df9"
        },
        {
          "@id": "_:N85933204bd0f4e978b69b4066eb6ca06"
        }
      ],
      "sh:targetClass": {
        "@id": "http://example.org/type/prompt-template"
      }
    },
    {
      "@id": "_:N056ddae112d243938488a3369ef22df9",
      "sh:datatype": {
        "@id": "xsd:string"
      },
      "sh:maxCount": 1,
      "sh:minCount": 1,
      "sh:path": {
        "@id": "schema:name"
      }
    },
    {
      "@id": "_:N85933204bd0f4e978b69b4066eb6ca06",
      "sh:datatype": {
        "@id": "xsd:string"
      },
     

## Validation

Validates the validity of the graph against the defined SHACL shapes.

### Does it conform?

In [10]:
conforms, results_graph, results_text = validate(g)
conforms

True

### Validation results

In [11]:
jsonld = results_graph.serialize(format="json-ld", indent=2, auto_compact=True)
print(jsonld)

{
  "@context": {
    "owl": "http://www.w3.org/2002/07/owl#",
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "schema": "https://schema.org/",
    "sh": "http://www.w3.org/ns/shacl#",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@id": "_:Nb89dc8c471a84394b6775a7d4a6f4868",
  "@type": "sh:ValidationReport",
  "sh:conforms": true
}
